In [8]:
import os
os.chdir("../")
%pwd

'c:\\Users\\91885\\Desktop\\Text_summarization'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir :Path
    data_path : Path
    model_ckpt :Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int 
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str 
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int    

In [10]:
from textSummarizer.utils.common import read_yaml , create_directories
from textSummarizer.constants import *



In [11]:
class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath =PARAM_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        param = self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
                root_dir = config.root_dir,
                data_path  = config.data_path,
                model_ckpt  = config.model_ckpt,
                num_train_epochs = param.num_train_epochs ,
                warmup_steps = param.warmup_steps ,
                per_device_train_batch_size = param.per_device_train_batch_size  ,
                per_device_eval_batch_size = param.per_device_eval_batch_size ,
                weight_decay = param.weight_decay ,
                logging_steps = param.logging_steps ,
                evaluation_strategy = param.evaluation_strategy  ,
                eval_steps = param.eval_steps ,
                save_steps = param.save_steps ,
                gradient_accumulation_steps = param.gradient_accumulation_steps 
            )
        
        return model_trainer_config
    

In [12]:
from transformers import TrainingArguments , Trainer ,DataCollatorForSeq2Seq , AutoModelForSeq2SeqLM , AutoTokenizer 
from datasets import load_dataset , load_from_disk
import torch

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
        output_dir=self.config.root_dir,
        num_train_epochs=self.config.num_train_epochs,
        warmup_steps=self.config.warmup_steps,
        per_device_train_batch_size=self.config.per_device_train_batch_size,
        per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        weight_decay=self.config.weight_decay,
        logging_steps=self.config.logging_steps,
        eval_strategy=self.config.evaluation_strategy,  
        eval_steps=self.config.eval_steps,
        save_steps=1e6,
        gradient_accumulation_steps=self.config.gradient_accumulation_steps
         )


        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [14]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e :
    raise e   

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'